<a href="https://colab.research.google.com/github/yashs087/hate_speech_detection/blob/master/shailja_mam_hate_dataset1(21022020).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
#mport pickle
#import sys
from sklearn.feature_extraction.text import TfidfVectorizer #vectorizer to change and test
import nltk #natural language tool kit
from nltk.stem.porter import *
import string
import re #regular expression
#from textstat.textstat import *
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [3]:
data= pd.read_csv("/content/dataset1.csv")
data.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [4]:
len(data)

24783

In [5]:
set(data["class"])

{0, 1, 2}

In [0]:
list(data["Label"]).count(0)

2898

In [0]:
list(data["Label"]).count(1)

1049

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
tweets = data.tweet

stopwords=stopwords = nltk.corpus.stopwords.words("english")  #removing stopwords for better performance

stemmer = PorterStemmer() #algorithm for for removing the common morphological and inflexional endings from words in English.

def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
                       '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    parsed_text = re.sub(space_pattern, ' ', text_string)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    return parsed_text

def tokenize(tweet):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems tweets. Returns a list of stemmed tokens."""
    tweet = " ".join(re.split("[^a-zA-Z]*", tweet.lower())).strip()
    tokens = [stemmer.stem(t) for t in tweet.split() if t not in stopwords and len(t)> 2]
    tokens = [t for t in tokens if len(t)>2]
    return tokens

In [0]:
vectorizer = TfidfVectorizer( #An alternative is to calculate word frequencies, and by far the most popular method is called TF-IDF
    tokenizer=tokenize,
    preprocessor=preprocess,
    ngram_range=(1, 3), #an n-gram is a contiguous sequence of n items from a given sample of text
    stop_words=stopwords,
    use_idf=True,
    smooth_idf=False,
    norm=None,
    decode_error='replace',
    max_features=10000,
    min_df=5,
    max_df=0.75
    )

In [9]:
tfidf=vectorizer.fit_transform(tweets).toarray()

/usr/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [0]:
x = tfidf
y = data['class'].astype(int)

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.35)

In [0]:
classifiers = [
    LogisticRegression(),
    LinearSVC(),
    GaussianNB(),
    SGDClassifier(),
    KNeighborsClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    MLPClassifier()
]

In [0]:
def df(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split('      ')
        row['class'] = row_data[0]
        row['precision'] = float(row_data[1])
        row['recall'] = float(row_data[2])
        row['f1_score'] = float(row_data[3])
        row['support'] = float(row_data[4])
        report_data.append(row)
    dataframe = pd.DataFrame.from_dict(report_data)
    return dataframe

In [14]:
Results=[]
for i,item in enumerate(classifiers):
    clf = item
    clf.fit(X_train, y_train)
    print(item,'\n')
    y_preds = clf.predict(X_test)
    report = classification_report( y_test, y_preds,output_dict=True)
    Results.append(report)
    print(report,"\n")
    print(confusion_matrix(y_test, y_preds),"\n\n\n")


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False) 

{'0': {'precision': 0.27230046948356806, 'recall': 0.23673469387755103, 'f1-score': 0.25327510917030566, 'support': 490}, '1': {'precision': 0.9000144279324773, 'recall': 0.9281356940931409, 'f1-score': 0.9138587752710225, 'support': 6721}, '2': {'precision': 0.8042488619119879, 'recall': 0.7240437158469946, 'f1-score': 0.762041696621136, 'support': 1464}, 'accuracy': 0.8546397694524496, 'macro avg': {'precision': 0.6588545864426777, 'recall': 0.6296380346058955, 'f1-score': 0.6430585270208214, 'support': 8675}, 'weighted avg': {'precision': 0.8483970644403779, 'recall': 0.8546397694524496, 'f1-score': 0.8509254957859753, 'support': 8675}} 

[[ 116  335   39]
 [ 2

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0) 

{'0': {'precision': 0.19814814814814816, 'recall': 0.21836734693877552, 'f1-score': 0.20776699029126214, 'support': 490}, '1': {'precision': 0.8884345794392523, 'recall': 0.9052224371373307, 'f1-score': 0.8967499447269511, 'support': 6721}, '2': {'precision': 0.7544677544677545, 'recall': 0.6632513661202186, 'f1-score': 0.7059251181388586, 'support': 1464}, 'accuracy': 0.8255907780979828, 'macro avg': {'precision': 0.613683494018385, 'recall': 0.5956137167321083, 'f1-score': 0.6034806843856907, 'support': 8675}, 'weighted avg': {'precision': 0.8268359877284841, 'recall': 0.8255907780979828, 'f1-score': 0.8256295765657459, 'support': 8675}} 

[[ 107  343   40]
 [ 361 6084  276]
 [  72  421  971]] 



GaussianNB(priors=None, var_smoothing=1e-09) 

{'0': {'prec

In [0]:
lr=pd.DataFrame(Results[0]).transpose()
svc=pd.DataFrame(Results[1]).transpose()
gnb=pd.DataFrame(Results[2]).transpose()
sgd=pd.DataFrame(Results[3]).transpose()
knn=pd.DataFrame(Results[4]).transpose()
tree=pd.DataFrame(Results[5]).transpose()
forest=pd.DataFrame(Results[6]).transpose()
mlp=pd.DataFrame(Results[7]).transpose()

In [0]:
result = pd.concat([lr,svc,gnb,sgd,knn,tree,forest,mlp], keys=['LogisticRegression','LinearSVC','GaussianNB','SGDClassifier','KNeighborsClassifier','DecisionTreeClassifier','RandomForestClassifier','MLPClassifier'])

In [17]:
result

precision    recall  f1-score      support
LogisticRegression     0              0.272300  0.236735  0.253275   490.000000
                       1              0.900014  0.928136  0.913859  6721.000000
                       2              0.804249  0.724044  0.762042  1464.000000
                       accuracy       0.854640  0.854640  0.854640     0.854640
                       macro avg      0.658855  0.629638  0.643059  8675.000000
                       weighted avg   0.848397  0.854640  0.850925  8675.000000
LinearSVC              0              0.198148  0.218367  0.207767   490.000000
                       1              0.888435  0.905222  0.896750  6721.000000
                       2              0.754468  0.663251  0.705925  1464.000000
                       accuracy       0.825591  0.825591  0.825591     0.825591
                       macro avg      0.613683  0.595614  0.603481  8675.000000
                       weighted avg   0.826836  0.825591  0.825630  8675.000000
GaussianNB             0              0.097606  0.432653  0.159279   490.000000
                       1              0.892722  0.666121  0.762952  6721.000000
                       2              0.550403  0.559426  0.554878  1464.000000
                       accuracy       0.634928  0.634928  0.634928     0.634928
                       macro avg      0.513577  0.552733  0.492369  8675.000000
                       weighted avg   0.790040  0.634928  0.693739  8675.000000
SGDClassifier          0              0.263285  0.222449  0.241150   490.000000
                       1              0.897082  0.928582  0.912560  6721.000000
                       2              0.793712  0.706967  0.747832  1464.000000
                       accuracy       0.851297  0.851297  0.851297     0.851297
                       macro avg      0.651360  0.619333  0.633848  8675.000000
                       weighted avg   0.843838  0.851297  0.846837  8675.000000
KNeighborsClassifier   0              0.468599  0.197959  0.278336   490.000000
                       1              0.861052  0.955215  0.905692  6721.000000
                       2              0.775692  0.536202  0.634087  1464.000000
                       accuracy       0.841729  0.841729  0.841729     0.841729
                       macro avg      0.701781  0.563125  0.606038  8675.000000
                       weighted avg   0.824479  0.841729  0.824420  8675.000000
DecisionTreeClassifier 0              0.321053  0.248980  0.280460   490.000000
                       1              0.906376  0.934831  0.920384  6721.000000
                       2              0.810712  0.754781  0.781747  1464.000000
                       accuracy       0.865706  0.865706  0.865706     0.865706
                       macro avg      0.679380  0.646197  0.660864  8675.000000
                       weighted avg   0.857170  0.865706  0.860842  8675.000000
RandomForestClassifier 0              0.484211  0.187755  0.270588   490.000000
                       1              0.911531  0.955066  0.932791  6721.000000
                       2              0.820513  0.808743  0.814585  1464.000000
                       accuracy       0.887032  0.887032  0.887032     0.887032
                       macro avg      0.738751  0.650521  0.672655  8675.000000
                       weighted avg   0.872034  0.887032  0.875438  8675.000000
MLPClassifier          0              0.308475  0.185714  0.231847   490.000000
                       1              0.896347  0.941824  0.918523  6721.000000
                       2              0.799697  0.719945  0.757728  1464.000000
                       accuracy       0.861671  0.861671  0.861671     0.861671
                       macro avg      0.668173  0.615828  0.636033  8675.000000
                       weighted avg   0.846830  0.861671  0.852601  8675.000000